## Data Source:
https://docs.google.com/spreadsheets/d/16Oi7F6RZh6w1mSO821RW6UzqWE-oSExZDHLhRzcK5Ug/edit?fbclid=IwAR0zF8xkZYIBzj74aLYKc5SvkIGXey5OY1DlSVOSducLX3fnLaeMY2wzX8w#gid=2028039576

## install chart_studio

In [ ]:
%%sh
pip install chart_studio

In [ ]:
from chart_studio.plotly import plot, iplot

In [ ]:
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
import plotly.graph_objects as go
 

daily_stats = pd.read_csv('https://od.cdc.gov.tw/eic/Day_Confirmation_Age_County_Gender_19CoV.csv')
daily_stats['個案研判日_datetime'] = pd.to_datetime(daily_stats['個案研判日']).dt.date

daily_stats.rename(columns= {'個案研判日':'個案研判日_string'}, inplace=True)

daily_stats['年齡層'] = daily_stats['年齡層'].replace({'0':'00-04',
                           '1':'00-04',
                           '2':'00-04',
                           '3':'00-04',
                           '4':'00-04',
                           '5-9':'05-09'})

daily_stats['年齡層'] = daily_stats['年齡層'].str.replace('-', '~')

date_start = datetime.date(2022, 4, 1)
mask = (daily_stats.個案研判日_datetime > date_start) & (daily_stats.確定病名 == '嚴重特殊傳染性肺炎')
daily_stats = daily_stats[mask]

mask = (daily_stats.性別 == '男') | (daily_stats.性別 == '女')
daily_stats = daily_stats[mask]

daily_stats['是否為境外移入'].replace('None', '否', inplace=True)

In [ ]:
daily_stats

In [ ]:
# 4月起，各縣市每日確診趨勢

pivot1 = daily_stats.pivot_table(index=['個案研判日_string','縣市'], values='確定病例數', aggfunc='sum')
pivot1 = pivot1.reset_index()
mask = pivot1.縣市 != '空值'
pivot1 = pivot1[mask]

fig1 = px.bar(pivot1, y='縣市', x='確定病例數',
         orientation='h',
         animation_frame='個案研判日_string',
         title='縣市確診病例數')
fig1.update_layout(yaxis_categoryorder = 'total ascending')

fig1.show()

In [ ]:
# 境外移入與比例

from plotly.subplots import make_subplots

pivot2 = daily_stats.pivot_table(index='個案研判日_datetime', columns='是否為境外移入', values='確定病例數', aggfunc=sum)
pivot2 = pivot2.reset_index()

pivot2['境外移入比例(%)'] = pivot2.eval('是 / (是+否)*100')

pivot2

fig2 = make_subplots(specs=[[{"secondary_y": True}]])

fig2.add_trace(go.Bar(name='境外移入案例', x=pivot2['個案研判日_datetime'], y=pivot2['是']),
        secondary_y=False)


fig2.add_trace(
  go.Scatter(name='境外移入比例(%)', x=pivot2['個案研判日_datetime'], y=pivot2['境外移入比例(%)']),
  secondary_y=True)

fig2.update_layout(title = '境外移入案例與境外移入比例', yaxis_range=[0, 250])
fig2.update_yaxes(range=[0, 50], secondary_y=True)

fig2.show()

In [ ]:
pivot2

In [ ]:
# 本土七日均線

mask = daily_stats.是否為境外移入 == '否'
local_daily = daily_stats[mask]
pivot3 = local_daily.pivot_table(index='個案研判日_datetime', values=['確定病例數'], aggfunc='sum')

pivot3['本土七日均線'] = pivot3.rolling(window=7, min_periods=1).mean().round(2)

pivot3

fig3 = go.Figure(data=[
  go.Bar(name='本土案例', x=pivot3.index, y=pivot3['確定病例數']),
  go.Scatter(name='本土七日均線', x=pivot3.index, y=pivot3['本土七日均線'])
])
fig3.update_layout(title = '本土七日均線')

fig3.show()


In [ ]:
# 確診年齡層

pivot4 = daily_stats.pivot_table(index=['個案研判日_datetime','年齡層','性別'], values=['確定病例數'], aggfunc='sum')
pivot4 = pivot4.reset_index()

#test_df = pivot4[ pivot4['個案研判日_datetime'] == datetime.date(2022, 4, 2) ]


male  = pivot4[pivot4['性別'] == '男']
female = pivot4[pivot4['性別'] == '女']

layout = go.Layout(yaxis=go.layout.YAxis(title='年齡'))

fig4 = go.Figure(data=[
    go.Bar(name='男',
        y=male['年齡層'],
        x=male['確定病例數'],
        orientation='h'),

    go.Bar(name='女',
        y=female['年齡層'],
        x=female['確定病例數'].apply(lambda x: -x),
        orientation='h')
])

fig4.update_layout(barmode='relative',
         title='確診年齡層分布-1')


fig4.show()


In [ ]:
# 確診年齡層

pivot4 = daily_stats.pivot_table(index=['個案研判日_string','年齡層','性別'], values=['確定病例數'], aggfunc='sum')
pivot4 = pivot4.reset_index()

fig5 = px.bar(pivot4, x="年齡層", y='確定病例數', barmode='group', color='確定病例數',
              facet_row='性別',
              hover_data=['個案研判日_string','性別','年齡層','確定病例數'],
              title='確診年齡層分布-2')




fig5.show()


In [ ]:
# 確診年齡層

pivot4 = daily_stats.pivot_table(index=['個案研判日_string','年齡層','性別'], values=['確定病例數'], aggfunc='sum')
pivot4 = pivot4.reset_index()

fig6 = px.bar(pivot4, x="年齡層", y='確定病例數', barmode='group', color='確定病例數',
              facet_row='性別',
              hover_data=['個案研判日_string','性別','年齡層','確定病例數'],
              title='確診年齡層分布-3',
              animation_frame='個案研判日_string')




fig6.show()


In [ ]:
%%sh
pip install jupyter-dash -q
pip install dash-cytoscape -q

In [ ]:
from jupyter_dash import JupyterDash  # pip install dash
import dash_cytoscape as cyto  # pip install dash-cytoscape==0.2.0 or higher
import dash_html_components as html
from dash import dcc
from dash.dependencies import Output, Input

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div(children=[
    html.H1(children='Covid-19 Analysis'),

    html.Div(children=dcc.Graph(id='fig1', figure=fig1)),
    html.Div(children=dcc.Graph(id='fig2', figure=fig2)),
    html.Div(children=dcc.Graph(id='fig3', figure=fig3)),
    html.Div(children=dcc.Graph(id='fig4', figure=fig4)),
    html.Div(children=dcc.Graph(id='fig5', figure=fig5)),
    html.Div(children=dcc.Graph(id='fig6', figure=fig6)),

    

    

    
])

if __name__ == '__main__':
    app.run_server(debug=True)
    app.run_server(mode='inline', port=8030)

## Resources
dash app gallery
1. https://dash.gallery/Portal/

dash-pivottable
1. https://github.com/plotly/dash-pivottable

2. https://dash.gallery/dash-pivottable/